Encoders:
 - https://huggingface.co/monsoon-nlp/hindi-bert
 - https://huggingface.co/flax-community/roberta-hindi
 - https://huggingface.co/vasista22/whisper-hindi-small
 
Decoders:
- https://huggingface.co/csebuetnlp/mT5_m2o_hindi_crossSum
- https://huggingface.co/docs/transformers/model_doc/openai-gpt
- https://huggingface.co/docs/transformers/model_doc/ctrl
- https://huggingface.co/docs/transformers/model_doc/gpt2

In [82]:
# from transformers import EncoderDecoderModel, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("monsoon-nlp/hindi-bert")
# model = EncoderDecoderModel.from_encoder_decoder_pretrained("monsoon-nlp/hindi-bert", "gpt2")

# model.config.decoder_start_token_id = tokenizer.cls_token_id
# model.config.pad_token_id = tokenizer.pad_token_id

# # a hindi sentence
# input_ids = tokenizer(
#     'भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष राहुल गांधी ने शनिवार को कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया है।',
#     return_tensors="pt",
# ).input_ids

# labels = tokenizer(
#     'राहुल गांधी ने कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया है।', return_tensors="pt",
# ).input_ids

# # the forward function automatically creates the correct decoder_input_ids
# outputs = model(input_ids=input_ids, labels=labels)

In [83]:
# outputs = model.generate(input_ids, decoder_start_token_id=model.config.decoder_start_token_id)
# outputs

In [84]:
# tokenizer2 = AutoTokenizer.from_pretrained("gpt2")
# tokenizer2.decode(outputs[0], skip_special_tokens=True, predict_with_generate=True)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

test_df = pd.read_csv('/raid/home/dhruv/pankaji/NLP_PROJECT/Data/HindiNews_test.txt')
train_df = pd.read_csv('/raid/home/dhruv/pankaji/NLP_PROJECT/Data/hindi_train.txt')
print(train_df.shape, test_df.shape)

train_df = train_df.dropna()
test_df = test_df.dropna()

train_val_split = 0.15
train_df, val_df = train_test_split(train_df, test_size=train_val_split, random_state=42)
print(train_df.shape, val_df.shape, test_df.shape)

(21225, 4) (3000, 3)
(18041, 4) (3184, 4) (3000, 3)


In [2]:
train_df.head()

,Id,Heading,Summary,Article
8713,hindi_2023_train_8713,प्रोडक्शन के लिए एम्ब्रेयर और सुखोई की बातचीत ...,india looks partner sukhoi embraer to manufact...,भारत सरकार दूर-दराज इलाकों से बेहतर एयर कनेक्ट...
19043,hindi_2022_9186,UP: भदोही में नशे में धुत युवक ने महात्मा गांध...,जिसके बाद कांग्रेस कार्यकर्ता एकत्रित होकर गां...,भदोहीः उत्तर प्रदेश के भदोही में बीती शनिवार र...
17257,hindi_2022_7400,भारत के इस क्षेत्र में नहीं है Coronavirus का ...,भारत में कोविड-19 के मामले 97.35 लाख के पार चल...,कोच्चि: भारत में कोविड-19 के मामले 97.35 लाख क...
3156,hindi_2023_train_3156,12वीं में पासिंग परसेंटेज के आधार पर परीक्षा म...,NEET 2024 - No Minimum Marks Needed For NEET. ...,12वीं के बाद मेडिकल की पढ़ाई करने वाले छात्रों ...
15583,hindi_2022_5726,"उत्तर प्रदेश में कोरोना वायरस से 17 और मौतें, ...",उत्‍तर प्रदेश में शुक्रवार को कोरोना वायरस संक...,लखनऊ: उत्‍तर प्रदेश में शुक्रवार को कोरोना वाय...


In [3]:
import torch
from transformers import EncoderDecoderModel, AutoTokenizer, AutoModelForCausalLM,AutoModelForSeq2SeqLM, MT5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum") #"aashay96/indic-gpt"
#model = EncoderDecoderModel.from_encoder_decoder_pretrained("ai4bharat/IndicBART","facebook/mbart-large-en-ro" )


decoder_model = MT5ForConditionalGeneration.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum").get_decoder()
#decoder_model.enc_to_dec_proj=torch.nn.Linear(768, len(tokenizer))
#decoder_model.enc_to_dec_proj.name = "lm_head"
device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')

encoder_model =  AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum").get_encoder()
model = EncoderDecoderModel.from_encoder_decoder_pretrained(encoder_model=encoder_model, decoder_pretrained_model_name_or_path="ai4bharat/IndicBART")
model.config.decoder_start_token_id = tokenizer.pad_token_id#tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
tokenizer.add_tokens(["<sum>"], special_tokens=True )
model = model.to(device)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/raid/home/dhruv/miniconda3/envs/myenv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
train = iter(train_df['Heading'].iloc[i]+train_df['Article'].iloc[i] for i in range(len(train_df)))
new_tokenizer = tokenizer.train_new_from_iterator(train, vocab_size=10000)
new_tokens = set(new_tokenizer.vocab) - set(tokenizer.vocab)


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [5]:
len(new_tokenizer.vocab)

10000

In [6]:
tokenizer.add_tokens(list(new_tokens))

8170

In [7]:
model.decoder.resize_token_embeddings(len(tokenizer))
#model.resize_token_embeddings(len(tokenizer))
model.encoder.resize_token_embeddings(len(tokenizer))


Embedding(258271, 768)

In [8]:
model

EncoderDecoderModel(
  (encoder): MT5Stack(
    (embed_tokens): Embedding(258271, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_fea

In [9]:
import torch

In [10]:
import torch.nn as nn
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 3

In [11]:
def train(model, tokenizer, train_df, optimizer):
    model.train()
    total_loss = 0
    batch = 0
    batch_loss = 0
    batch_size = 1000
    for i, row in train_df.iterrows():
        input = tokenizer("<sum>"+row['Heading']+row["Article"], return_tensors="pt")
        input_ids = input.input_ids.to(device)
        attention_mask = input.attention_mask.to(device)
        labels = tokenizer(row['Summary'], return_tensors="pt")
        labels_input_ids = labels.input_ids.to(device)
        labels_attention_mask = labels.attention_mask.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,  labels=labels_input_ids, decoder_attention_mask=labels_attention_mask)
        loss = outputs.loss
        total_loss += loss.item()
        batch_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        batch += 1
        if batch % batch_size == 0:
            print(f"Batch {batch} Loss: {batch_loss / batch_size}")
            batch_loss = 0
    print(f"Training Loss: {total_loss / len(train_df)}")
        

def evaluate(model, tokenizer, val_df):
    model.eval()
    total_loss = 0
    for i, row in val_df.iterrows():
        input = tokenizer("<sum>"+row['Heading']+row["Article"], return_tensors="pt")
        input_ids = input.input_ids.to(device)
        attention_mask = input.attention_mask.to(device)
        labels = tokenizer(row['Summary'], return_tensors="pt")
        labels_input_ids = labels.input_ids.to(device)
        labels_attention_mask = labels.attention_mask.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,  labels=labels_input_ids, decoder_attention_mask=labels_attention_mask)
        total_loss += outputs.loss.item()
    print(f"Validation Loss: {total_loss / len(val_df)}")


for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    train(model, tokenizer, train_df, optimizer)
    evaluate(model, tokenizer, val_df)

Epoch 1


/raid/home/dhruv/miniconda3/envs/myenv/lib/python3.11/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:636: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


OutOfMemoryError: CUDA out of memory. Tried to allocate 496.00 MiB. GPU 6 has a total capacity of 39.59 GiB of which 466.19 MiB is free. Process 3570645 has 2.84 GiB memory in use. Process 1768747 has 1.00 GiB memory in use. Process 1797221 has 1.00 GiB memory in use. Process 1839624 has 1.00 GiB memory in use. Process 3957216 has 1.00 GiB memory in use. Process 3962250 has 1.00 GiB memory in use. Process 3985294 has 1.00 GiB memory in use. Process 3995523 has 1.00 GiB memory in use. Process 2482300 has 5.06 GiB memory in use. Including non-PyTorch memory, this process has 24.21 GiB memory in use. Of the allocated memory 22.18 GiB is allocated by PyTorch, and 732.04 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [23]:
head_text = test_df['Heading'][12]
article_text = test_df["Article"][12]
print(test_df['id'][6])
print(text)
input_ids = tokenizer("<sum>"+text, return_tensors="pt", truncation=True).input_ids.to(device)
print(input_ids.shape)

outputs = model.generate(input_ids, decoder_start_token_id=model.config.decoder_start_token_id, num_beams=4)
print(outputs)
# tokenizer2
summary = tokenizer.decode(outputs[0], skip_special_tokens=True, predict_with_generate=True)
print("Summary:", summary)

HindiNews_test_6
करियर बेस्ट पारी में मंधाना ने जमाए दादा जैसे चौके-छक्के, 87 रन बनाए
torch.Size([1, 23])
tensor([[     0,    364, 267182,  69670, 264657,    331,  15164, 322946, 264657,
            259,    280,    581, 306873,    259,    272,    581, 306873,    259,
            296,      2]], device='cuda:6')
Summary: Boll ywoo d Actressoo l Update n Update ;


In [29]:
test_df["Article"].iloc[3]
test_df["Heading"].iloc[3]
test_df.columns

Index(['id', 'Article', 'Heading'], dtype='object')